# Testing how to implement Piecewise Layers

## TODO:
1. Figure out how to do batching with this code
2. Ensure this is the most efficient
3. Polish up the mess

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
# Use eager execution to make debugging easier (causes tensorflow to evaluate functions immediately)
tf.enable_eager_execution()

## Piecewise Linear

In [3]:
# Create bin array with nbins, each with a width of 1.0/nbins. 
# Note: the bins array contains the bin edges including 0, which makes finding alpha easier later on
nbins = 32
width = 1.0/nbins
bins = np.array([[x*width for x in range(nbins+1)] for i in range(3)])

# Print bins for testing purposes
bins

array([[0.     , 0.03125, 0.0625 , 0.09375, 0.125  , 0.15625, 0.1875 ,
        0.21875, 0.25   , 0.28125, 0.3125 , 0.34375, 0.375  , 0.40625,
        0.4375 , 0.46875, 0.5    , 0.53125, 0.5625 , 0.59375, 0.625  ,
        0.65625, 0.6875 , 0.71875, 0.75   , 0.78125, 0.8125 , 0.84375,
        0.875  , 0.90625, 0.9375 , 0.96875, 1.     ],
       [0.     , 0.03125, 0.0625 , 0.09375, 0.125  , 0.15625, 0.1875 ,
        0.21875, 0.25   , 0.28125, 0.3125 , 0.34375, 0.375  , 0.40625,
        0.4375 , 0.46875, 0.5    , 0.53125, 0.5625 , 0.59375, 0.625  ,
        0.65625, 0.6875 , 0.71875, 0.75   , 0.78125, 0.8125 , 0.84375,
        0.875  , 0.90625, 0.9375 , 0.96875, 1.     ],
       [0.     , 0.03125, 0.0625 , 0.09375, 0.125  , 0.15625, 0.1875 ,
        0.21875, 0.25   , 0.28125, 0.3125 , 0.34375, 0.375  , 0.40625,
        0.4375 , 0.46875, 0.5    , 0.53125, 0.5625 , 0.59375, 0.625  ,
        0.65625, 0.6875 , 0.71875, 0.75   , 0.78125, 0.8125 , 0.84375,
        0.875  , 0.90625, 0.9375 , 0.968

In [4]:
# randomly assign unnormalized values to piecewise PDF, here we will be using a total of 6 dimensions (|B|=3)
Q = np.random.rand(3,nbins)

# normalize the piecewise PDF such that the integral is 1
total = np.sum(Q,axis=1)
Q[0]=Q[0]/total[0]
Q[1]=Q[1]/total[1]
Q[2]=Q[2]/total[2]

# Insert a 0 to the beginning to match the size to the bin array
Q = np.insert(Q,0,0,axis=1)

# Print out values of Q for testing purposes
Q

array([[0.        , 0.0478476 , 0.05587782, 0.01583139, 0.01568572,
        0.02375241, 0.06753198, 0.01031194, 0.01673977, 0.0170834 ,
        0.00764915, 0.04362138, 0.07064042, 0.00438733, 0.04242122,
        0.01226465, 0.00863941, 0.04076193, 0.01310798, 0.03401279,
        0.05850288, 0.00245296, 0.02893618, 0.0446203 , 0.02408887,
        0.05106999, 0.01242378, 0.04034815, 0.05196614, 0.04011281,
        0.03079363, 0.02469377, 0.04182225],
       [0.        , 0.00669279, 0.02120551, 0.02494908, 0.0155497 ,
        0.04635294, 0.00928684, 0.01846544, 0.01550253, 0.00110668,
        0.03756745, 0.0496762 , 0.04298007, 0.00399215, 0.0157639 ,
        0.02645966, 0.01929507, 0.05857586, 0.04750707, 0.06165386,
        0.0608555 , 0.01399955, 0.04765035, 0.00156031, 0.05324194,
        0.05877753, 0.02187782, 0.02073061, 0.05640611, 0.03095775,
        0.0407378 , 0.03265758, 0.03796437],
       [0.        , 0.03279353, 0.03303719, 0.04569255, 0.03213382,
        0.04669561, 0.0330

In [5]:
# Generate a single phase space point in xB
xB = np.random.rand(1,3)

# Print out xB for testing purposes
xB

array([[0.13004505, 0.56898751, 0.14241574]])

In [21]:
# searchsorted finds the location that each xB should be inserted into bins in order to keep the bins array sorted
# The transpose is needed to make sure that each point is compared to the right dimension
b=tf.searchsorted(bins,tf.transpose(xB),side='right')

# The result returns a transpose of a bin location array, take the transpose and print for testing
tf.transpose(b).numpy()

array([[ 5, 19,  5]], dtype=int32)

In [7]:
# Test to ensure that the correct Q values are pulled out from the arrays
# Note: that each dimension pulls out each bin, there may be a way around this, 
#       but the desired result is on the diagonal
Q[:,b[:,0]]

array([[0.02375241, 0.03401279, 0.02375241],
       [0.04635294, 0.06165386, 0.04635294],
       [0.04669561, 0.00032921, 0.04669561]])

In [20]:
# This calculates the PDF for the Piecewise Linear layer, 
# it takes the result from above and takes the product of the diagonal.
# Again there may be a better way to do this
tf.reduce_prod(tf.diag_part(Q[:,b[:,0]])).numpy()

6.838235442790042e-05

In [14]:
# Calculate the alpha value in each dimension. 
# Note: the -1 in the bins term, this is to use the left edge of the bin instead of the right edge
alpha = (xB-tf.diag_part(bins[:,b[:,0]-1]))/width

# Print out the values of alpha for testing
alpha.numpy()

array([[0.16144171, 0.20760045, 0.55730355]])

In [15]:
# Calculates the first part of the coupling layer term (\alpha Q_{ib}) in each dimension
tf.diag_part(alpha*Q[:,b[:,0]]).numpy()

array([0.00383463, 0.01279937, 0.02602363])

In [16]:
# Calculates the second part of the coupling layer term (sum_{k=1}^{b-1} Q_{ik}) in each dimension
# There may be a more efficient way of doing this.
tf.diag_part(np.cumsum(Q,axis=1)[:,b[:,0]]).numpy()

array([0.15899494, 0.52258279, 0.19035271])

In [19]:
# Calculates the full coupling layer terms
(tf.diag_part(alpha*Q[:,b[:,0]])+tf.diag_part(np.cumsum(Q,axis=1)[:,b[:,0]])).numpy()

array([0.16282957, 0.53538216, 0.21637634])